In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Definindo estilo para visualizações
plt.style.use('ggplot')
sns.set_palette('Set2')

# Carregando os dados (substitua pelo caminho real do arquivo)
def load_data(file_path):
    try:
        # Tentando diferentes formatos dependendo da extensão
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(file_path)
        else:
            raise ValueError("Formato de arquivo não suportado")
        
        print(f"Dados carregados com sucesso: {df.shape[0]} linhas e {df.shape[1]} colunas")
        return df
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

# Função para limpeza de dados
def clean_data(df):
    print("Iniciando limpeza de dados...")
    
    # Cópia para não modificar o original
    df_clean = df.copy()
    
    # Renomeando colunas para remover underscores e padronizar
    df_clean.columns = [col.replace('_', '') if col != 'mobile_phone_use_for_education' else 'education_use_frequency' 
                        for col in df_clean.columns]
    
    # Removendo a linha com muitos valores ausentes (Mehvish)
    df_clean = df_clean[df_clean['names'] != 'Mehvish']
    
    # Tratando valores ausentes
    # Para colunas categóricas, preenchemos com o valor mais comum
    categorical_cols = ['gender', 'mobileoperatingsystem', 'education_use_frequency', 
                        'mobilephoneactivities', 'helpfulforstudying', 'educationalapps',
                        'performanceimpact', 'usagedistraction', 'attentionspan',
                        'usefulfeatures', 'healthrisks', 'beneficialsubject',
                        'usagesymptoms', 'symptomfrequency', 'healthprecautions', 'healthrating']
    
    for col in categorical_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])
    
    # Convertendo idade para categoria ordenada
    age_order = ['16-20', '21-25', '26-30', '31-35']
    df_clean['age'] = pd.Categorical(df_clean['age'], categories=age_order, ordered=True)
    
    # Tratando múltiplas respostas em colunas (separadas por ponto e vírgula)
    multi_response_cols = ['mobilephoneactivities', 'usagesymptoms', 'healthrating']
    for col in multi_response_cols:
        if col in df_clean.columns:
            # Criando colunas dummy para cada resposta possível
            if df_clean[col].dtype == 'object':
                df_clean[col] = df_clean[col].str.split(';')
    
    # Limpando a coluna de tempo de uso diário
    if 'dailyusages' in df_clean.columns:
        # Ordenando os valores de tempo de uso
        usage_order = ['< 2 hours', '2-4 hours', '4-6 hours', '> 6 hours']
        df_clean['dailyusages'] = pd.Categorical(df_clean['dailyusages'], categories=usage_order, ordered=True)
    
    print(f"Limpeza concluída. Dimensões finais: {df_clean.shape[0]} linhas e {df_clean.shape[1]} colunas")
    return df_clean

# Função para análise exploratória
def exploratory_analysis(df):
    print("Iniciando análise exploratória...")
    
    # Estatísticas demográficas
    print("\n=== PERFIL DEMOGRÁFICO ===")
    print("\nDistribuição por Idade:")
    age_dist = df['age'].value_counts(normalize=True).sort_index() * 100
    print(age_dist.round(1).astype(str) + '%')
    
    print("\nDistribuição por Gênero:")
    gender_dist = df['gender'].value_counts(normalize=True) * 100
    print(gender_dist.round(1).astype(str) + '%')
    
    # Padrões de uso
    print("\n=== PADRÕES DE USO ===")
    print("\nSistema Operacional:")
    os_dist = df['mobileoperatingsystem'].value_counts(normalize=True) * 100
    print(os_dist.round(1).astype(str) + '%')
    
    print("\nTempo de Uso Diário:")
    usage_dist = df['dailyusages'].value_counts(normalize=True).sort_index() * 100
    print(usage_dist.round(1).astype(str) + '%')
    
    # Atividades principais (tratando múltiplas respostas)
    print("\nPrincipais Atividades:")
    # Simplificando para categorias principais
    def simplify_activities(act):
        if isinstance(act, list):
            if 'All of these' in act:
                return 'All of these'
            elif len(act) > 1:
                return 'Multiple activities'
            elif len(act) == 1:
                return act[0]
        elif isinstance(act, str):
            if act == 'All of these':
                return 'All of these'
            elif ';' in act:
                return 'Multiple activities'
            else:
                return act
        return 'Not specified'
    
    df['simplified_activities'] = df['mobilephoneactivities'].apply(simplify_activities)
    activities_dist = df['simplified_activities'].value_counts(normalize=True) * 100
    print(activities_dist.round(1).astype(str) + '%')
    
    # Uso educacional
    print("\n=== USO EDUCACIONAL ===")
    print("\nFrequência de Uso para Educação:")
    edu_freq_dist = df['education_use_frequency'].value_counts(normalize=True) * 100
    print(edu_freq_dist.round(1).astype(str) + '%')
    
    print("\nAplicativos Educacionais Mais Utilizados:")
    edu_apps_dist = df['educationalapps'].value_counts(normalize=True) * 100
    print(edu_apps_dist.round(1).astype(str) + '%')
    
    print("\nPercepção de Utilidade para Estudos:")
    helpful_dist = df['helpfulforstudying'].value_counts(normalize=True) * 100
    print(helpful_dist.round(1).astype(str) + '%')
    
    # Impacto no desempenho
    print("\n=== IMPACTO NO DESEMPENHO ===")
    print("\nImpacto no Desempenho Acadêmico:")
    performance_dist = df['performanceimpact'].value_counts(normalize=True) * 100
    print(performance_dist.round(1).astype(str) + '%')
    
    print("\nMomentos de Distração:")
    distraction_dist = df['usagedistraction'].value_counts(normalize=True) * 100
    print(distraction_dist.round(1).astype(str) + '%')
    
    print("\nImpacto na Capacidade de Atenção:")
    attention_dist = df['attentionspan'].value_counts(normalize=True) * 100
    print(attention_dist.round(1).astype(str) + '%')
    
    # Consequências para a saúde
    print("\n=== CONSEQUÊNCIAS PARA A SAÚDE ===")
    
    # Simplificando sintomas para categorias principais
    def simplify_symptoms(symp):
        if isinstance(symp, list):
            if 'All of these' in symp:
                return 'All of these'
            elif len(symp) > 1:
                return 'Multiple symptoms'
            elif len(symp) == 1:
                return symp[0]
        elif isinstance(symp, str):
            if symp == 'All of these':
                return 'All of these'
            elif ';' in symp:
                return 'Multiple symptoms'
            else:
                return symp
        return 'None reported'
    
    df['simplified_symptoms'] = df['usagesymptoms'].apply(simplify_symptoms)
    
    print("\nSintomas Relatados pelo Uso:")
    symptoms_dist = df['simplified_symptoms'].value_counts(normalize=True) * 100
    print(symptoms_dist.round(1).astype(str) + '%')
    
    print("\nFrequência dos Sintomas:")
    symptom_freq_dist = df['symptomfrequency'].value_counts(normalize=True) * 100
    print(symptom_freq_dist.round(1).astype(str) + '%')
    
    print("\nPrecauções de Saúde Adotadas:")
    precautions_dist = df['healthprecautions'].value_counts(normalize=True) * 100
    print(precautions_dist.round(1).astype(str) + '%')
    
    # Simplificando avaliação de saúde
    def simplify_health_rating(rating):
        if isinstance(rating, list):
            return rating[0]  # Pegamos apenas a primeira avaliação
        elif isinstance(rating, str):
            if ';' in rating:
                return rating.split(';')[0]  # Pegamos apenas a primeira avaliação
            else:
                return rating
        return 'Not specified'
    
    df['simplified_health'] = df['healthrating'].apply(simplify_health_rating)
    
    print("\nAutoavaliação da Saúde:")
    health_dist = df['simplified_health'].value_counts(normalize=True) * 100
    print(health_dist.round(1).astype(str) + '%')
    
    # Recursos e percepções
    print("\n=== RECURSOS E PERCEPÇÕES ===")
    print("\nRecursos Considerados Mais Úteis:")
    features_dist = df['usefulfeatures'].value_counts(normalize=True) * 100
    print(features_dist.round(1).astype(str) + '%')
    
    print("\nPercepção sobre Riscos à Saúde:")
    risks_dist = df['healthrisks'].value_counts(normalize=True) * 100
    print(risks_dist.round(1).astype(str) + '%')
    
    print("\nÁreas Acadêmicas Beneficiadas:")
    subjects_dist = df['beneficialsubject'].value_counts(normalize=True) * 100
    print(subjects_dist.round(1).astype(str) + '%')
    
    print("\nAnálise exploratória concluída.")
    return df

# Função para visualizações
def create_visualizations(df):
    print("Gerando visualizações...")
    
    # Configurando o tamanho da figura
    plt.figure(figsize=(12, 10))
    
    # 1. Gráfico de barras para distribuição de idade
    plt.subplot(2, 2, 1)
    sns.countplot(x='age', data=df, order=df['age'].value_counts().index)
    plt.title('Distribuição por Faixa Etária')
    plt.xlabel('Faixa Etária')
    plt.ylabel('Contagem')
    
    # 2. Gráfico de pizza para gênero
    plt.subplot(2, 2, 2)
    gender_counts = df['gender'].value_counts()
    plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribuição por Gênero')
    
    # 3. Gráfico de barras para tempo de uso diário
    plt.subplot(2, 2, 3)
    sns.countplot(x='dailyusages', data=df, order=['< 2 hours', '2-4 hours', '4-6 hours', '> 6 hours'])
    plt.title('Tempo de Uso Diário')
    plt.xlabel('Horas de Uso')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    
    # 4. Gráfico de barras para sintomas relatados
    plt.subplot(2, 2, 4)
    sns.countplot(y='simplified_symptoms', data=df, order=df['simplified_symptoms'].value_counts().index[:6])
    plt.title('Principais Sintomas Relatados')
    plt.xlabel('Contagem')
    plt.ylabel('Sintomas')
    
    plt.tight_layout()
    plt.savefig('uso_celular_demografico.png', dpi=300)
    
    # Segunda figura: Uso educacional e impacto
    plt.figure(figsize=(12, 10))
    
    # 1. Gráfico de barras para frequência de uso educacional
    plt.subplot(2, 2, 1)
    sns.countplot(x='education_use_frequency', data=df, 
                 order=['Frequently', 'Sometimes', 'Rarely', 'Never'])
    plt.title('Frequência de Uso para Educação')
    plt.xlabel('Frequência')
    plt.ylabel('Contagem')
    
    # 2. Gráfico de barras para aplicativos educacionais
    plt.subplot(2, 2, 2)
    sns.countplot(y='educationalapps', data=df, order=df['educationalapps'].value_counts().index[:5])
    plt.title('Aplicativos Educacionais Mais Utilizados')
    plt.xlabel('Contagem')
    plt.ylabel('Tipo de Aplicativo')
    
    # 3. Gráfico de barras para impacto no desempenho
    plt.subplot(2, 2, 3)
    performance_order = ['Strongly agree', 'Agree', 'Neutral', 'Disagree', 'Strongly disagree']
    sns.countplot(x='performanceimpact', data=df, order=performance_order)
    plt.title('Impacto no Desempenho Acadêmico')
    plt.xlabel('Nível de Concordância')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    
    # 4. Gráfico de barras para momentos de distração
    plt.subplot(2, 2, 4)
    distraction_order = ['During Class Lectures', 'While Studying', 'During Exams', 'Not Distracting']
    sns.countplot(y='usagedistraction', data=df, order=distraction_order)
    plt.title('Momentos de Distração')
    plt.xlabel('Contagem')
    plt.ylabel('Momento')
    
    plt.tight_layout()
    plt.savefig('uso_celular_educacional.png', dpi=300)
    
    # Terceira figura: Saúde e precauções
    plt.figure(figsize=(12, 10))
    
    # 1. Gráfico de barras para frequência de sintomas
    plt.subplot(2, 2, 1)
    freq_order = ['Frequently', 'Sometimes', 'Rarely', 'Never']
    sns.countplot(x='symptomfrequency', data=df, order=freq_order)
    plt.title('Frequência dos Sintomas')
    plt.xlabel('Frequência')
    plt.ylabel('Contagem')
    
    # 2. Gráfico de barras para precauções de saúde
    plt.subplot(2, 2, 2)
    precaution_order = df['healthprecautions'].value_counts().index[:4]
    sns.countplot(y='healthprecautions', data=df, order=precaution_order)
    plt.title('Precauções de Saúde Adotadas')
    plt.xlabel('Contagem')
    plt.ylabel('Precaução')
    
    # 3. Gráfico de barras para autoavaliação de saúde
    plt.subplot(2, 2, 3)
    health_order = ['Excellent', 'Good', 'Fair', 'Poor']
    sns.countplot(x='simplified_health', data=df, order=health_order)
    plt.title('Autoavaliação da Saúde')
    plt.xlabel('Avaliação')
    plt.ylabel('Contagem')
    
    # 4. Gráfico de barras para percepção de riscos à saúde
    plt.subplot(2, 2, 4)
    risks_order = ['Yes', 'Only Partially', 'No']
    sns.countplot(x='healthrisks', data=df, order=risks_order)
    plt.title('Percepção sobre Riscos à Saúde')
    plt.xlabel('Percepção')
    plt.ylabel('Contagem')
    
    plt.tight_layout()
    plt.savefig('uso_celular_saude.png', dpi=300)
    
    print("Visualizações salvas como arquivos PNG.")

# Função para análise de correlações
def correlation_analysis(df):
    print("\nAnalisando correlações entre variáveis...")
    
    # Convertendo variáveis categóricas para numéricas para análise de correlação
    df_corr = df.copy()
    
    # Mapeamento para frequência de uso educacional
    edu_freq_map = {'Frequently': 4, 'Sometimes': 3, 'Rarely': 2, 'Never': 1}
    df_corr['education_use_freq_num'] = df_corr['education_use_frequency'].map(edu_freq_map)
    
    # Mapeamento para tempo de uso diário
    usage_map = {'< 2 hours': 1, '2-4 hours': 2, '4-6 hours': 3, '> 6 hours': 4}
    df_corr['daily_usage_num'] = df_corr['dailyusages'].map(usage_map)
    
    # Mapeamento para impacto no desempenho
    performance_map = {'Strongly agree': 5, 'Agree': 4, 'Neutral': 3, 'Disagree': 2, 'Strongly disagree': 1}
    df_corr['performance_impact_num'] = df_corr['performanceimpact'].map(performance_map)
    
    # Mapeamento para frequência de sintomas
    symptom_freq_map = {'Frequently': 4, 'Sometimes': 3, 'Rarely': 2, 'Never': 1}
    df_corr['symptom_freq_num'] = df_corr['symptomfrequency'].map(symptom_freq_map)
    
    # Mapeamento para avaliação de saúde
    health_map = {'Excellent': 4, 'Good': 3, 'Fair': 2, 'Poor': 1}
    df_corr['health_rating_num'] = df_corr['simplified_health'].map(health_map)
    
    # Selecionando apenas colunas numéricas para correlação
    numeric_cols = ['education_use_freq_num', 'daily_usage_num', 'performance_impact_num', 
                    'symptom_freq_num', 'health_rating_num']
    
    # Calculando correlação
    corr_matrix = df_corr[numeric_cols].corr()
    
    # Visualizando matriz de correlação
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0,
                xticklabels=['Uso Educacional', 'Tempo de Uso', 'Impacto Desempenho', 
                             'Freq. Sintomas', 'Avaliação Saúde'],
                yticklabels=['Uso Educacional', 'Tempo de Uso', 'Impacto Desempenho', 
                             'Freq. Sintomas', 'Avaliação Saúde'])
    plt.title('Matriz de Correlação entre Variáveis')
    plt.tight_layout()
    plt.savefig('correlacao_uso_celular.png', dpi=300)
    
    print("Análise de correlação concluída e salva como imagem.")
    
    # Principais insights de correlação
    print("\nPrincipais insights de correlação:")
    
    # Encontrando as correlações mais fortes (positivas e negativas)
    corr_unstack = corr_matrix.unstack()
    corr_unstack = corr_unstack[corr_unstack < 1.0]  # Removendo auto-correlações (sempre 1.0)
    
    # Top 3 correlações positivas
    top_pos = corr_unstack.nlargest(3)
    print("\nCorrelações positivas mais fortes:")
    for idx, val in top_pos.items():
        print(f"{idx[0]} e {idx[1]}: {val:.2f}")
    
    # Top 3 correlações negativas
    top_neg = corr_unstack.nsmallest(3)
    print("\nCorrelações negativas mais fortes:")
    for idx, val in top_neg.items():
        print(f"{idx[0]} e {idx[1]}: {val:.2f}")

# Função principal
def main():
    # Caminho do arquivo (substitua pelo caminho real)
    file_path = "saudevscelular.csv"
    
    # Carregando dados
    df = load_data(file_path)
    
    if df is not None:
        # Limpeza de dados
        df_clean = clean_data(df)
        
        # Análise exploratória
        df_analyzed = exploratory_analysis(df_clean)
        
        # Visualizações
        create_visualizations(df_analyzed)
        
        # Análise de correlações
        correlation_analysis(df_analyzed)
        
        # Salvando dados limpos
        df_clean.to_csv("dados_uso_celular_limpos.csv", index=False)
        print("\nDados limpos salvos como 'dados_uso_celular_limpos.csv'")
        
        print("\nAnálise completa finalizada com sucesso!")

if __name__ == "__main__":
    main()

Dados carregados com sucesso: 100 linhas e 20 colunas
Iniciando limpeza de dados...


KeyError: 'names'